In [1]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [4]:
##data ingestion to vector db pipleine

##reading all pdf inside directory
def read_docs (pdf_directory):
    all_docs=[]
    pdf_dir=Path(pdf_directory)


    pdf_files=list(pdf_dir.glob('**/*pdf'))

    print(f'found{ len(pdf_files)}to process')

    for pdf_file in pdf_files:
        print(f"\n Processing :{pdf_file.name}")
        try:
            loader=PyMuPDFLoader(str(pdf_file))
            documents=loader.load() 

            for d in documents:
                d.metadata['source_file'] = pdf_file.name
                d.metadata['file_type']='pdf' 

            all_docs.extend(documents)   
            print(f" loaded {len(documents)} pages")

        except Exception as e:
            print(f" Error: {e}")
    
    print(f"\n Total Documents loaded : {len(all_docs)}")
    return all_docs


read_docs_all = read_docs("/Users/sai/Projects/mlprojects/notebooks/data/pdfs")

found5to process

 Processing :Policy-on-Materiality-of-RPT.pdf
 loaded 5 pages

 Processing :Remuneration-Policy-for-Directors.pdf
 loaded 5 pages

 Processing :Policy-for-Selection-of-Directors.pdf
 loaded 5 pages

 Processing :CSR-Policy.pdf
 loaded 13 pages

 Processing :RIL_Anti-Bribery-and-Anti-Corruption-Policy.pdf
 loaded 11 pages

 Total Documents loaded : 39


In [5]:
read_docs_all

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-16T14:38:18+05:30', 'source': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'file_path': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': 'Policy on Materiality of RPT copy', 'author': 'Balasaheb Wagaralkar', 'subject': '', 'keywords': '', 'moddate': '2025-10-16T14:38:18+05:30', 'trapped': '', 'modDate': "D:20251016143818+05'30'", 'creationDate': "D:20251016143818+05'30'", 'page': 0, 'source_file': 'Policy-on-Materiality-of-RPT.pdf', 'file_type': 'pdf'}, page_content='POLICY ON MATERIALITY OF \nRELATED PARTY TRANSACTIONS AND \nON DEALING WITH \nRELATED PARTY TRANSACTIONS'),
 Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-16T14:38:18+05:30', 'source': '/Users/sai/Projects/mlproject

In [10]:
#text splitting gwt into chunks


def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Spit documents iinto smaller chunks"""
    text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    separators=["\n\n","\n"," ",""]
    )

    split_docs=text_splitter.split_documents(documents)
    print(f"split{len(documents)} documetns into {len(split_docs)} chunks")

    if split_docs:
        print(f"\nExample Chunk")
        print(f"Content:{split_docs[0].page_content[:200]}...")
        print(f"Metadata:{split_docs[0].metadata}")

    return split_docs

In [11]:
chunks=split_documents(read_docs_all)

split39 documetns into 92 chunks

Example Chunk
Content:POLICY ON MATERIALITY OF 
RELATED PARTY TRANSACTIONS AND 
ON DEALING WITH 
RELATED PARTY TRANSACTIONS...
Metadata:{'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-16T14:38:18+05:30', 'source': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'file_path': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': 'Policy on Materiality of RPT copy', 'author': 'Balasaheb Wagaralkar', 'subject': '', 'keywords': '', 'moddate': '2025-10-16T14:38:18+05:30', 'trapped': '', 'modDate': "D:20251016143818+05'30'", 'creationDate': "D:20251016143818+05'30'", 'page': 0, 'source_file': 'Policy-on-Materiality-of-RPT.pdf', 'file_type': 'pdf'}


In [12]:
chunks

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-16T14:38:18+05:30', 'source': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'file_path': '/Users/sai/Projects/mlprojects/notebooks/data/pdfs/Policy-on-Materiality-of-RPT.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': 'Policy on Materiality of RPT copy', 'author': 'Balasaheb Wagaralkar', 'subject': '', 'keywords': '', 'moddate': '2025-10-16T14:38:18+05:30', 'trapped': '', 'modDate': "D:20251016143818+05'30'", 'creationDate': "D:20251016143818+05'30'", 'page': 0, 'source_file': 'Policy-on-Materiality-of-RPT.pdf', 'file_type': 'pdf'}, page_content='POLICY ON MATERIALITY OF \nRELATED PARTY TRANSACTIONS AND \nON DEALING WITH \nRELATED PARTY TRANSACTIONS'),
 Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-16T14:38:18+05:30', 'source': '/Users/sai/Projects/mlproject

Embedding

In [14]:
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
from chromadb.config import Settings
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity
 

In [17]:
class EmbeddingManager:

    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        """Load the semtentce transformer"""
        try:
            print(f"Loading the embedding model : {self.model_name}")
            self.model=SentenceTransformer(self.model_name)
            print(f"Model loaded successfully . embedding dimensions :{ self.model.get_sentence_embedding_dimension()}")

        except Exception as e:
            print(f"Error in handling model {self.model_name}:{e}")
            raise

    def gen_emebeddings(self,texts:List[str])-> np.ndarray:
        """Generate embeddings for a list of text"""

        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating emebddings for {len(texts)} texts...")
        embeddings=self.model.encode(texts,show_progress_bar=True)
        return embeddings
    

In [18]:
###initalize emebddings manager


embeddingManager=EmbeddingManager()
embeddingManager

Loading the embedding model : all-MiniLM-L6-v2
Model loaded successfully . embedding dimensions :384
